In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from io import BytesIO
import numpy as np
import pandas as pd
import re
import pickle
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from io import BytesIO
from project_lib import Project


project = Project(sc,"6a9ddb2e-72df-4e1b-bba5-c1764c25c533","p-ccf0395c1a40d8af007b1009245f2892bc8e300c")


client_29882ce9c0f74130ab5806368b60732b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fr5UusEaEba2L3BgsHv_p91SAwsntg4rwsdUk8y51MMd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_29882ce9c0f74130ab5806368b60732b.get_object(Bucket='trainclassifier-donotdelete-pr-avgynhohutocd8', Key='tfidf.pickle')['Body']
streaming_body_2 = client_29882ce9c0f74130ab5806368b60732b.get_object(Bucket='trainclassifier-donotdelete-pr-avgynhohutocd8', Key='svcclassifier.pickle')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 
readrawdata1 = streaming_body_1.read()
readrawdata2= streaming_body_2.read()

tfidf_transformer = pickle.load(BytesIO(readrawdata1))
classifier = pickle.load(BytesIO(readrawdata2))


body = client_29882ce9c0f74130ab5806368b60732b.get_object(Bucket='trainclassifier-donotdelete-pr-avgynhohutocd8',Key='category_id.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
category_id_df = pd.read_csv(body)
category_to_id = dict(category_id_df.values)
body = client_29882ce9c0f74130ab5806368b60732b.get_object(Bucket='trainclassifier-donotdelete-pr-avgynhohutocd8',Key='Defect_Summary.csv')['Body']
predict_value_df = pd.read_csv(body)



def transform_row(data):
  review = re.sub('[^a-zA-Z]', ' ', data)
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  new_corpus = [review]
  new_X_test = tfidf_transformer.transform(new_corpus).toarray()
  return new_X_test


def pred_resolution(row):
 pred_value=classifier.predict(row)
 return (category_to_id[pred_value[0]])



predict_value_df['transformed_row']=predict_value_df['summary'].apply(lambda s : transform_row(s))
predict_value_df['predicted_resolution']=predict_value_df['transformed_row'].apply(lambda s : pred_resolution(s))
predict_value_df.drop(['transformed_row'],axis=1,inplace=True)
project.save_data(file_name = "predicted_out.csv",data = predict_value_df.to_csv(index=False),overwrite=True)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200906070419-0002
KERNEL_ID = aa080ffc-e443-4593-b49c-6fb03e2a5051


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark/shared/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'file_name': 'predicted_out.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'trainclassifier-donotdelete-pr-avgynhohutocd8',
 'asset_id': 'af95b813-fb2e-496d-9e9e-b91a1c284da2'}